# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier,RadiusNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB, MultinomialNB


/home/enzo/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/enzo/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/enzo/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/enzo/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


[nltk_data] Downloading package punkt to /home/enzo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/enzo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
# load data from database
engine = create_engine('sqlite:///disaster.db')
df = pd.read_sql_table('InsertTableName', con=engine)

relevant_categories = df.columns[4:]

X = df['message']
y = df[relevant_categories]

In [5]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X.head(5)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [7]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    
    # Replace any URL with  urlplaceholder
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # remove punctuation and Convert to lowercase
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
        
    tokens = word_tokenize(text)
    
    # Remove stop-words
    stop_words = stopwords.words("english")
    tokens = [token for token in tokens if token not in stop_words]
    
    # lemmitize
    lemmatizer = nltk.WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens
tokenize("testing: tested I do not doesn't lovVed loves loving you warnings .... http://www.hotmail.com")

['testing', 'tested', 'lovved', 'love', 'loving', 'warning', 'urlplaceholder']

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
        
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

#### Evaluation for the train set 

In [9]:
y_train_pred = pipeline.predict(X_train)
print(classification_report(y_train, y_train_pred, target_names=relevant_categories))

                        precision    recall  f1-score   support

               related       0.99      0.99      0.99     14893
               request       1.00      0.93      0.96      3299
                 offer       1.00      0.70      0.83        81
           aid_related       0.99      0.97      0.98      8114
          medical_help       1.00      0.85      0.92      1559
      medical_products       1.00      0.84      0.91       994
     search_and_rescue       1.00      0.78      0.88       543
              security       1.00      0.72      0.84       350
              military       1.00      0.86      0.93       651
                 water       1.00      0.90      0.95      1268
                  food       1.00      0.96      0.98      2191
               shelter       1.00      0.91      0.95      1703
              clothing       1.00      0.86      0.92       325
                 money       1.00      0.79      0.88       446
        missing_people       1.00      

#### Evaluation for the test set 

In [10]:
y_test_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_test_pred, target_names=relevant_categories))

                        precision    recall  f1-score   support

               related       0.86      0.90      0.88      5013
               request       0.80      0.43      0.56      1175
                 offer       0.00      0.00      0.00        37
           aid_related       0.73      0.60      0.66      2746
          medical_help       0.55      0.10      0.17       525
      medical_products       0.68      0.09      0.16       319
     search_and_rescue       0.67      0.08      0.14       181
              security       0.00      0.00      0.00       121
              military       0.52      0.07      0.13       209
                 water       0.88      0.21      0.34       404
                  food       0.84      0.57      0.68       732
               shelter       0.76      0.31      0.44       611
              clothing       0.71      0.12      0.21        80
                 money       0.75      0.04      0.07       158
        missing_people       1.00      

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = {'vect__min_df': [1, 5, 10],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25, 50], 
              'clf__estimator__min_samples_split':[2, 5, 10, 20, 40]}

cv = GridSearchCV(pipeline, param_grid = parameters, scoring = 'f1_weighted', verbose = 1, cv=3)

In [12]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 90 candidates, totalling 270 fits


/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/s

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/s

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/s

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/s

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/s

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/s

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/s

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/s

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__min_df': [1, 5, 10], 'tfidf__use_idf': [True, False], 'clf__estimator__n_estimators': [10, 25, 50], 'clf__estimator__min_samples_split': [2, 5, 10, 20, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_weighted', verbose=1)

In [13]:
cv.best_params_

{'clf__estimator__min_samples_split': 5,
 'clf__estimator__n_estimators': 50,
 'tfidf__use_idf': True,
 'vect__min_df': 10}

{'clf__estimator__min_samples_split': 10,
 'clf__estimator__n_estimators': 50,
 'tfidf__use_idf': False,
 'vect__min_df': 10}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
best_model_predict=cv.predict(X_test)

In [15]:
print(classification_report(y_test, best_model_predict, target_names=relevant_categories))

                        precision    recall  f1-score   support

               related       0.86      0.92      0.89      5013
               request       0.83      0.51      0.63      1175
                 offer       0.00      0.00      0.00        37
           aid_related       0.73      0.72      0.73      2746
          medical_help       0.59      0.16      0.25       525
      medical_products       0.70      0.22      0.34       319
     search_and_rescue       0.65      0.11      0.19       181
              security       0.25      0.01      0.02       121
              military       0.60      0.11      0.19       209
                 water       0.83      0.54      0.66       404
                  food       0.82      0.72      0.77       732
               shelter       0.80      0.52      0.63       611
              clothing       0.82      0.17      0.29        80
                 money       0.82      0.06      0.11       158
        missing_people       0.00      

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [16]:
pickle.dump(cv, open('multiforest.sav', 'wb'))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [9]:
def test_pipeline(pipeline):
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    pipeline.fit(X_train, y_train)
    y_test_pred = pipeline.predict(X_test)
    print(classification_report(y_test, y_test_pred, target_names=relevant_categories))

#### LinearSVC

In [10]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(LinearSVC(multi_class="crammer_singer"), n_jobs=1))
        
    ])

test_pipeline(pipeline)  

                        precision    recall  f1-score   support

               related       0.88      0.91      0.89      5021
               request       0.77      0.60      0.67      1127
                 offer       0.00      0.00      0.00        34
           aid_related       0.74      0.70      0.72      2776
          medical_help       0.62      0.28      0.38       562
      medical_products       0.60      0.28      0.38       327
     search_and_rescue       0.76      0.21      0.33       182
              security       0.40      0.02      0.03       129
              military       0.56      0.32      0.41       230
                 water       0.77      0.66      0.71       423
                  food       0.79      0.74      0.76       740
               shelter       0.75      0.57      0.65       544
              clothing       0.78      0.51      0.62       111
                 money       0.54      0.24      0.33       164
        missing_people       0.58      

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
k=LinearSVC(multi_class="crammer_singer")

In [14]:
k.get_params

<bound method BaseEstimator.get_params of LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='crammer_singer', penalty='l2', random_state=None,
     tol=0.0001, verbose=0)>

#### LogisticRegressionCV(multi_class="multinomial")

In [19]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(LogisticRegressionCV(multi_class="multinomial")))
        #('clf', RandomForestClassifier())
    ])

test_pipeline(pipeline)  

                        precision    recall  f1-score   support

               related       0.85      0.94      0.89      4993
               request       0.94      0.15      0.25      1112
                 offer       0.00      0.00      0.00        27
           aid_related       0.90      0.27      0.41      2690
          medical_help       0.75      0.01      0.01       517
      medical_products       1.00      0.01      0.01       307
     search_and_rescue       0.00      0.00      0.00       191
              security       0.00      0.00      0.00       109
              military       0.59      0.11      0.18       220
                 water       0.60      0.02      0.04       439
                  food       0.87      0.08      0.14       724
               shelter       0.86      0.04      0.08       563
              clothing       1.00      0.02      0.04       106
                 money       0.00      0.00      0.00       163
        missing_people       0.00      

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#### MultinomialNB

In [20]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(MultinomialNB()))
        
    ])

test_pipeline(pipeline) 

                        precision    recall  f1-score   support

               related       0.78      0.99      0.87      5002
               request       0.85      0.21      0.33      1119
                 offer       0.00      0.00      0.00        37
           aid_related       0.73      0.61      0.66      2726
          medical_help       0.00      0.00      0.00       533
      medical_products       1.00      0.00      0.01       345
     search_and_rescue       0.00      0.00      0.00       189
              security       0.00      0.00      0.00       112
              military       0.00      0.00      0.00       220
                 water       0.00      0.00      0.00       380
                  food       0.83      0.03      0.05       714
               shelter       0.00      0.00      0.00       562
              clothing       0.00      0.00      0.00        98
                 money       0.00      0.00      0.00       166
        missing_people       0.00      

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#### DecisionTreeClassifier

In [21]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
        
    ])

test_pipeline(pipeline) 

                        precision    recall  f1-score   support

               related       0.86      0.86      0.86      5005
               request       0.57      0.53      0.55      1156
                 offer       0.00      0.00      0.00        24
           aid_related       0.64      0.64      0.64      2749
          medical_help       0.38      0.33      0.35       523
      medical_products       0.46      0.37      0.41       334
     search_and_rescue       0.24      0.21      0.23       197
              security       0.09      0.08      0.08       132
              military       0.39      0.33      0.36       226
                 water       0.62      0.61      0.62       384
                  food       0.73      0.74      0.74       755
               shelter       0.62      0.55      0.58       565
              clothing       0.62      0.58      0.60       107
                 money       0.40      0.32      0.36       150
        missing_people       0.25      

#### AdaBoostClassifier

In [22]:
from sklearn.ensemble import AdaBoostClassifier
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
        
    ])

test_pipeline(pipeline) 

                        precision    recall  f1-score   support

               related       0.79      0.98      0.87      4965
               request       0.76      0.55      0.64      1156
                 offer       0.00      0.00      0.00        23
           aid_related       0.77      0.61      0.68      2717
          medical_help       0.57      0.26      0.36       542
      medical_products       0.65      0.33      0.44       316
     search_and_rescue       0.55      0.19      0.29       190
              security       0.33      0.04      0.07       129
              military       0.67      0.26      0.37       217
                 water       0.78      0.63      0.70       424
                  food       0.80      0.66      0.72       702
               shelter       0.77      0.57      0.66       566
              clothing       0.67      0.40      0.50       103
                 money       0.53      0.27      0.36       153
        missing_people       0.58      

#### SVC

In [23]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(SVC()))
        
    ])

test_pipeline(pipeline) 

                        precision    recall  f1-score   support

               related       0.75      1.00      0.86      4948
               request       0.00      0.00      0.00      1107
                 offer       0.00      0.00      0.00        31
           aid_related       0.00      0.00      0.00      2615
          medical_help       0.00      0.00      0.00       481
      medical_products       0.00      0.00      0.00       339
     search_and_rescue       0.00      0.00      0.00       173
              security       0.00      0.00      0.00       118
              military       0.00      0.00      0.00       216
                 water       0.00      0.00      0.00       434
                  food       0.00      0.00      0.00       715
               shelter       0.00      0.00      0.00       565
              clothing       0.00      0.00      0.00        93
                 money       0.00      0.00      0.00       155
        missing_people       0.00      

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Summary Performance for the tested Classifier Models (without gridsearch)

Model | precision |   recall | f1-score
------------ | -------------
RandomForestClassifier | 0.74  |    0.49  |    0.54 
LinearSVC |0.74  |    0.62 |     0.65
LogisticRegressionCV | 0.79 |     0.32    |  0.36
MultinomialNB |0.58 |     0.38  |    0.39 
DecisionTreeClassifier | 0.61  |    0.60 |     0.60
AdaBoostClassifier |0.73 |     0.59 |    0.63
SVC |0.18     | 0.24 |     0.21


since it is uneven distribution of cases for each set we look for the highest score for F1.
so we select the LinearSVC model.


In [15]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(LinearSVC(multi_class="crammer_singer"), n_jobs=1))
        
    ])
parameters = {
        'clf__estimator__C': [0.8, 1.0, 1.2, 1.4],
        'clf__estimator__max_iter': [500, 1000, 1200, 1500],
    } 
cv = GridSearchCV(pipeline, param_grid = parameters, scoring = 'f1_weighted', verbose = 1, cv=3)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
cv.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/s

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/enzo/anaconda3/lib/python3.6/s

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...rammer_singer', penalty='l2', random_state=None,
     tol=0.0001, verbose=0),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__C': [0.8, 1.0, 1.2, 1.4], 'clf__estimator__max_iter': [500, 1000, 1200, 1500]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_weighted', verbose=1)

In [17]:
cv.best_params_

{'clf__estimator__C': 1.2, 'clf__estimator__max_iter': 1000}

In [18]:
best_model_predict=cv.predict(X_test)
print(classification_report(y_test, best_model_predict, target_names=relevant_categories))

                        precision    recall  f1-score   support

               related       0.86      0.90      0.88      4987
               request       0.74      0.61      0.67      1133
                 offer       0.00      0.00      0.00        32
           aid_related       0.72      0.71      0.71      2701
          medical_help       0.57      0.33      0.42       510
      medical_products       0.69      0.30      0.42       351
     search_and_rescue       0.73      0.19      0.30       188
              security       0.27      0.03      0.05       120
              military       0.60      0.37      0.46       219
                 water       0.72      0.67      0.69       418
                  food       0.82      0.75      0.79       740
               shelter       0.78      0.59      0.67       597
              clothing       0.71      0.44      0.55        95
                 money       0.55      0.27      0.36       154
        missing_people       0.71      

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [19]:
pickle.dump(cv, open('LinearSVC.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.